## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges</span>
<sub>*And setting constants

In [110]:
import re
import json
import time
import random
import scrapy
import pickle
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import cloudscraper
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.request import urlopen
from fake_useragent import UserAgent
from scrapy.http import HtmlResponse
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.crawler import CrawlerProcess
from selenium.webdriver.common.by import By
from scrapy.utils.log import configure_logging
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [80]:
%matplotlib inline
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

proxies = {'http': 'http://109.254.67.104:9090',
           'http': 'http://103.49.202.252:80'}

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

In [77]:
data_jobs_titles = ['Data entry', 'Data engineer',
                    'Data scientist', 'Data analyst',
                    'ML developer']

indeed_countries = ['de', 'uk']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']

## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [6]:
def scrape_page_fast(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay= 10,browser= {
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False}) 
    
    content = scraper.get(url).text 
    return BeautifulSoup(content)



def scrape_page(url: str, retrieve_new_url= False) -> BeautifulSoup:
    
    chrome_options = Options()
    
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  
    
    driver = webdriver.Chrome(options= chrome_options)
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


    html = BeautifulSoup(driver.page_source)

    if retrieve_new_url:
        new_url = driver.current_url

        driver.quit()
        return html, new_url
        
    driver.quit()
    return html

In [7]:
# Comparing the two scrapers speed.
%timeit scrape_page_fast('https://linkedin.com')
%timeit scrape_page('https://linkedin.com')

# We can find that there's about 10X speed for the `scrape_page_fast` 🤯.

1.35 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.05 s ± 691 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# def indeed_scraper(page: int,
#                    country: str,
#                    job_title: str) -> BeautifulSoup:

#     modefid_page = (page - 1) * 10

#     data_job_title = job_title.replace(' ', '+')

#     # Pages in indeed.com are zero based indexed
#     url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
#     '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
#     '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
#     'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

#     soup = scrape_page(url)
#     loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

#     # available_pages = []

#     # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
#     #     available_pages.append(int(button.text))

#     if (int(page) != loaded_page):
#         # We will use KeyError as standard to represent getting out of the max pages.
#         raise KeyError
        
#     print(f'Page {page} Loaded successfully.')

#     return  soup

In [12]:
def linkedin_scraper(country:   str, # This scraper scrapes all pages at once
                     job_title: str,
                     page:      int) -> BeautifulSoup:
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
        'Accept-Encoding': '*',
        'Connection': 'keep-alive'}

    page: int = (page - 1) * 25
    url: str = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={job_title}&' + \
    f'location={country}&geoId=90000084&trk=public_jobs_j%20obs-search-bar_search-submit&position=1&pageNum=0&start={page}'

    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.content,'html.parser')

    return  soup

## <center><strong>Collecting<span style= 'color: #5cd3f7'> jobs </span> data

**What we will do in this section:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [13]:
def convert_to_integer(number_string):

    cleaned_string = ''.join(filter(str.isdigit, number_string))
    cleaned_string = cleaned_string.rstrip('+')
    integer_value = int(cleaned_string)

    return integer_value

In [63]:
soups = {'soup':        [],
         'job_title':   [],
         'country':     [],
         'platform':    [],
         'total_jobs':  []}

flag     : bool = False
max_page : int = 4 # TOREMOVE

# Linked In
print('Starting LinkedIn scraper.')

for country in linkedin_countries:

    print(f'Loading the {country} data.')
    for job_title in data_jobs_titles:
        
        full_soup   : str = ''
        total_jobs  : list = convert_to_integer(scrape_page(f'https://www.linkedin.com/jobs/search?keywords={job_title}&location={country}&' + 
                                                            f'\geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0').find(
                                                                   'span', 'results-context-header__job-count').text)
        
        for i in count(0):
            page = i + 1
            page_soup: BeautifulSoup = linkedin_scraper(country= country,
                                                        job_title= job_title,
                                                        page= i + page)
            full_soup += str(page_soup)
            full_soup += ' <br> '

            if (page_soup.find_all('li') is None) or (page == max_page):
                break

        soups['platform']      .append('LinkedIn')
        soups['total_jobs']    .append(total_jobs) 
        soups['soup']          .append(full_soup)
        soups['job_title']     .append(job_title)
        soups['country']       .append(country)

        print(f'Finished loading {job_title} jobs from the {country}.')
    print('\n')

Starting LinkedIn scraper.
Loading the European Union data.
Finished loading Data entry jobs from the European Union.
Finished loading Data engineer jobs from the European Union.
Finished loading Data scientist jobs from the European Union.
Finished loading Data analyst jobs from the European Union.
Finished loading ML developer jobs from the European Union.


Loading the United States data.
Finished loading Data entry jobs from the United States.
Finished loading Data engineer jobs from the United States.
Finished loading Data scientist jobs from the United States.
Finished loading Data analyst jobs from the United States.
Finished loading ML developer jobs from the United States.




In [15]:
# Indeed scraping TODO
# for country in indeed_countries:
#     for job_title in data_jobs_titles:
#         stacked_pages_soup: str = ''

#         for i in count(0):

#             try:
#                 page = i + 1
#                 soup: BeautifulSoup = indeed_scraper(country= country,
#                                                      job_title= job_title,
#                                                      page= page)

#                 stacked_pages_soup += '\n<br> ' + str(soup)
#                 if page > max_page:
#                     print('Finished loading current country or job title.\n')
#                     break

#             except KeyError as e:
#                 print('Finished loading current country or job title.\n')
#                 break

#         soups['platform'].append('Indeed')
#         soups['soup'].append(stacked_pages_soup)
#         soups['job_title'].append(job_title)
#         soups['country'].append(country)

#### **Collecting jobs & companies basic data**

In [16]:
def remove_escape_sequences(strings):

    new_strings = []
    escapes = ''.join([chr(char) for char in range(1, 32)])
    translator = str.maketrans('', '', escapes)

    for string in strings:
        new_string = string.translate(translator)
        new_strings.append(new_string)
        
    return tuple(new_strings)

In [66]:
soups = pd.DataFrame(soups)

soups['jobs_locations']    = np.nan
soups['listing_dates']     = np.nan
soups['jobs_titles']       = np.nan
soups['companies_names']   = np.nan

soups['companies_links']   = np.nan
soups['jobs_links']        = np.nan


# Collecting linkedIn jobs links
for i, soup in enumerate(soups[soups['platform'] == 'LinkedIn']['soup']):

    soup = BeautifulSoup(soup)

    linkedin_jobs_locations           = tuple([element.text for element in soup.find_all(
        'span', 'job-search-card__location')])

    linkedin_jobs_listing_dates       = tuple([element['datetime'] for element in soup.find_all(
        'time', 'job-search-card__listdate')])

    linkedin_jobs_titles              = tuple([element.text for element in soup.find_all(
        'h3', 'base-search-card__title')])

    linkedin_jobs_companies_names     = tuple([element.text for element in soup.find_all(
        'a', 'hidden-nested-link')])

    
    linkedin_jobs_links               = tuple([element['href'] for element in soup.find_all(
        'a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')])
    
    linkedin_companies_links          = tuple([element['href'] for element in soup.find_all(
        'a', 'hidden-nested-link')])

    # I will use sets and tuples instead of lists because they are faster.
    soups['jobs_locations']    [i] = remove_escape_sequences(linkedin_jobs_locations)
    soups['listing_dates']     [i] = linkedin_jobs_listing_dates
    soups['jobs_titles']       [i] = remove_escape_sequences(linkedin_jobs_titles)
    soups['companies_names']   [i] = remove_escape_sequences(linkedin_jobs_companies_names)

    soups['jobs_links']        [i] = linkedin_jobs_links
    soups['companies_links']   [i] = linkedin_companies_links

In [18]:
# Collecting indeed jobs links TOFIX
# for i, soup in enumerate(soups[soups['platform'] == 'Indeed']['soup']):
    
#     indeed_jobs_links: list = []
#     indeed_companies_links: list = []

#     for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
#         job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
#         indeed_jobs_links.append(job_link)

#     for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
#         company_link = BeautifulSoup(scrape_page(job_link)).find('a', 'css-775knl emf9s7v0')
#         indeed_companies_links.append(company_link)
#         print(company_link)
        
#     soups['total_jobs'][i] = len(indeed_jobs_links)
#     soups['jobs_links'][i] = indeed_jobs_links
#     soups['companies_links'][i] = indeed_companies_links

In [19]:

soups.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,companies_links,jobs_links
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,European Union,LinkedIn,305000,"( Fremont, CA , ...","(2023-07-14, 2023-07-14, 2023-07-14, 2023-07-1...",( Entry Level - Data Entry ...,( Get.It Recruit - Administrative ...,(https://www.linkedin.com/company/get-it-recru...,(https://www.linkedin.com/jobs/view/entry-leve...
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,European Union,LinkedIn,40000,"( San Ramon, CA , ...","(2023-06-15, 2023-04-07, 2023-04-28, 2023-05-2...",( Data Engineer ...,"( ShiftCode Analytics, Inc. ...",(https://www.linkedin.com/company/shiftcode-an...,(https://www.linkedin.com/jobs/view/data-engin...
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,European Union,LinkedIn,155000,"( San Francisco, CA , ...","(2023-06-16, 2023-07-10, 2023-07-06, 2023-07-1...",( Machine Learning Scientis...,"( Patterned Learning AI , ...",(https://www.linkedin.com/company/patterned-le...,(https://www.linkedin.com/jobs/view/machine-le...
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,European Union,LinkedIn,162000,"( San Francisco, CA , ...","(2023-02-20, 2023-05-11, 2023-05-08, 2023-01-2...",( Data Analyst ...,"( Digital Janet , ...",(https://www.linkedin.com/company/digitaljanet...,(https://www.linkedin.com/jobs/view/data-analy...
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,European Union,LinkedIn,1000,"( San Francisco, CA , ...","(2023-06-05, 2023-06-29, 2023-07-05, 2023-05-1...",( Engineer (Full Stack / ML...,"( Sieve , Auro...",(https://www.linkedin.com/company/sievedata?tr...,(https://www.linkedin.com/jobs/view/engineer-f...


#### **Loading more jobs data**

In [20]:
# Collecting more LinkedIn data

# We will collect the: 
#    describtion, credential category, location type
#
soups['describtion']         = None
soups['credential_category'] = None
soups['location_type']       = None
soups['employment_type']     = None
soups['industry']            = None
soups['country']             = None

for i, jobs in enumerate([soups['jobs_links'][0]]): # TOREMOVE remove the loop limit

    describtion:           list = []
    credential_category:   list = []
    location_type:         list = []
    employment_type:       list = []
    industry:              list = []
    country:               list = []
    
    for job in jobs:
            
        try:
            job_data:  dict = json.loads(scrape_page_fast(job).find(
                'script', attrs= {'type': 'application/ld+json'}).text)
        except:
            continue
            
        try: job_data['jobLocationType'] is True
        except: job_data['jobLocationType'] = np.nan
            
        try: job_data['applicantLocationRequirements'] is True
        except: job_data['applicantLocationRequirements'] = {'name': np.nan}

        try: job_data['educationRequirements'] is True
        except: job_data['educationRequirements'] = {'credentialCategory': np.nan}
        
        describtion               .append(job_data['description'].strip())
        credential_category       .append(job_data['educationRequirements']['credentialCategory'])
        location_type             .append(job_data['jobLocationType'])
        employment_type           .append(job_data['employmentType'].strip())
        industry                  .append(job_data['industry'].strip())
        country                   .append(job_data['applicantLocationRequirements']['name'])

    # Now we are going to convert list into tuples.
    soups['describtion']          [i] = tuple(describtion)
    soups['credential_category']  [i] = tuple(credential_category)
    soups['location_type']        [i] = tuple(location_type)
    soups['employment_type']      [i] = tuple(employment_type)
    soups['industry']             [i] = tuple(industry)
    soups['country']              [i] = tuple(country)

In [15]:
# TODO (loading indeed data.)

In [17]:
soups.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,companies_links,jobs_links,describtion,credential_category,location_type,employment_type,industry
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,"(United States, United States, United States, ...",LinkedIn,298000,"( Fremont, CA , ...","(2023-07-14, 2023-07-14, 2023-07-14, 2023-06-1...",( Entry Level - Data Entry ...,( Get.It Recruit - Administrative ...,(https://www.linkedin.com/company/get-it-recru...,(https://www.linkedin.com/jobs/view/entry-leve...,(Thank you for considering this opportunity! W...,"(high school, high school, high school, bachel...","(TELECOMMUTE, TELECOMMUTE, TELECOMMUTE, nan, T...","(FULL_TIME, FULL_TIME, FULL_TIME, PART_TIME, F...","(Human Resources Services, Human Resources Ser..."
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,None,LinkedIn,40000,"( San Ramon, CA , ...","(2023-06-15, 2023-04-07, 2023-04-28, 2023-05-2...",( Data Engineer ...,"( ShiftCode Analytics, Inc. ...",(https://www.linkedin.com/company/shiftcode-an...,(https://www.linkedin.com/jobs/view/data-engin...,None,None,None,None,None
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,None,LinkedIn,155000,"( San Francisco, CA , ...","(2023-06-16, 2023-07-10, 2023-07-17, 2023-05-0...",( Data Scientist ...,"( Experfy , Pa...",(https://www.linkedin.com/company/experfy?trk=...,(https://www.linkedin.com/jobs/view/data-scien...,None,None,None,None,None
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,None,LinkedIn,164000,"( San Francisco, CA , ...","(2023-02-20, 2023-05-11, 2023-05-08, 2023-01-2...",( Data Analyst ...,"( Digital Janet , ...",(https://www.linkedin.com/company/digitaljanet...,(https://www.linkedin.com/jobs/view/data-analy...,None,None,None,None,None
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,None,LinkedIn,1000,"( San Francisco, CA , ...","(2023-06-05, 2023-06-29, 2023-07-05, 2023-05-1...",( Engineer (Full Stack / ML...,"( Sieve , Auro...",(https://www.linkedin.com/company/sievedata?tr...,(https://www.linkedin.com/jobs/view/engineer-f...,None,None,None,None,None


#### **Loading more companies data**

In [40]:
def is_castable_to_number(s):
    return s.isdigit() or (s.isnumeric() if hasattr(s, 'isnumeric') else False)

In [119]:
get_random_proxy()

[{'_id': '63199fc9266407aa8c6935d6',
  'ip': '43.134.237.216',
  'anonymityLevel': 'elite',
  'asn': 'AS132203',
  'city': 'Singapore',
  'country': 'SG',
  'created_at': '2022-09-08T07:54:49.270Z',
  'google': False,
  'isp': 'Aceville Pte.ltd',
  'lastChecked': 1689758786,
  'latency': 393.68,
  'org': '',
  'port': '443',
  'protocols': ['socks5'],
  'region': None,
  'responseTime': 5392,
  'speed': 1,
  'updated_at': '2023-07-19T09:26:26.042Z',
  'workingPercent': None,
  'upTime': 38.79837067209776,
  'upTimeSuccessCount': 1143,
  'upTimeTryCount': 2946},
 {'_id': '630b6eb3de95dae521dc9c9e',
  'ip': '106.245.183.60',
  'anonymityLevel': 'elite',
  'asn': 'AS3786',
  'city': 'Pyeongtaek-si',
  'country': 'KR',
  'created_at': '2022-08-28T13:33:39.794Z',
  'google': False,
  'isp': 'LG DACOM Corporation',
  'lastChecked': 1689758786,
  'latency': 292.353,
  'org': 'Boranet',
  'port': '4145',
  'protocols': ['socks4'],
  'region': None,
  'responseTime': 4796,
  'speed': 84,
  'upd

In [113]:
for companies in [soups['companies_links'][0]]: # TOREMOVE

    company_descriptions   : list = []
    company_foundyears     : list = []
    company_interests      : list = []
    company_employees      : list = []
    company_followers      : list = []


    for company_url in set(companies): # We are using `set` type to elemenate the duplicates.

        ua = UserAgent()

        headers: dict = {
            'User-Agent': ua.random,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'}

        print(company_url)
        
        foundyear = None
        interest  = None

        # To repeat scraping the page if any thing unexpected happens
        company_url = company_url.split('?')[0]
        response    = requests.get(company_url,
                                   headers= headers,
                                   proxies= proxies)

        company_html = BeautifulSoup(response.content,'html.parser')
        company_data = company_html.find_all(
            'dd', class_='font-sans text-md text-color-text break-words overflow-hidden')

        print(str(company_html)[:100])
        
        try:
            followers = company_html.find(
                'p', class_= '!text-xs text-color-text-low-emphasis leading-[1.33333] m-0 truncate').text.strip()
        except:
            followers = None

        
        discription = company_html.find('meta', attrs= {'name': 'description'})['content']
        employees   = company_data[2].text.strip()

        if len(company_data) >= 6:
            if is_castable_to_number(company_data[5].text.strip()):
                foundyear = company_data[5].text.strip()
        
        if len(company_data) >= 7:
                interest = company_data[6].text.strip()


        company_descriptions   .append(discription)
        company_foundyears     .append(foundyear)
        company_interests      .append(interest)
        company_employees      .append(employees)
        company_followers      .append(followers)

https://www.linkedin.com/company/planet-forward?trk=public_jobs_jserp-result_job-search-card-subtitle
<html><head>
<script type="text/javascript">
var _0x26bb58=_0x3a5d;function _0x2511(){var _0x2338e3=


TypeError: 'NoneType' object is not subscriptable

In [102]:
company_interests

['Renewable: Wind and Solar, Nuclear and Fossil Power, Power Delivery: Transmission & Distribution, Power Generation, Civil & Environmental Engineering, Manufacturing & Light Industrial, Oil & Gas, Information Technology, HR/Administrative, Engineering, Automotive, Aerospace and Defense, Aviation, and Construction',
 'Teachers, Hiring, K-12, Education, Urban, Schools, Matching, Marketplace, parents, families, learningpod, remotelearning, learningpods, pandemicpods, and school leadership',
 'Staffing, Recruiting, Sourcing, Interviewing, Vetting, Candidate Assessment, Performance Assessment, Property Management, Sales, Sales Management, Frontline Management, Career Launchers, and Industry Builders',
 'Electric Vehicle Charging Solutions, Condominium Car Chargers, Electric Vehicle Supply Equipment (EVSE), Apartment Car Chargers, Fleet Charging, and EV Fleets',
 None,
 None,
 None,
 None,
 'Spanning the Stack from Firmware to the UI, Embedded Systems and Firmware Development, Managed Servi

## <center><strong><span style= 'color: #5cd3f7'>Saving</span> the data